In [2]:
import $file.^.Magic

import $file.$

In [3]:
Magic.!("pip", "install", "torch==2.4.*")
Magic.!("pip", "install", "tiktoken==0.7.*")


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [4]:
import $ivy.`dev.scalapy::scalapy-core:0.5.3`

import me.shadaj.scalapy.py
import py.SeqConverters
import py.PyQuote

val torch = py.module("torch")
val tiktoken = py.module("tiktoken")

import $ivy.$
import me.shadaj.scalapy.py
import py.SeqConverters
import py.PyQuote
torch: py.Module = <module 'torch' from '/usr/local/lib/python3.12/site-packages/torch/__init__.py'>
tiktoken: py.Module = <module 'tiktoken' from '/usr/local/lib/python3.12/site-packages/tiktoken/__init__.py'>

In [5]:
case class GPTConfig(
  vocabularySize: Int,
  contextLength: Int,
  embeddingDimension: Int,
  attentionHeadsCount: Int,
  layersCount: Int,
  dropoutRate: Double,
  queryKeyValueBias: Boolean
)

val gptConfig = GPTConfig(
  vocabularySize = 50_257,
  contextLength = 256,
  embeddingDimension = 768,
  attentionHeadsCount = 12,
  layersCount = 12,
  dropoutRate = 0.1,
  queryKeyValueBias = false
)

defined class GPTConfig
gptConfig: GPTConfig = GPTConfig(
  vocabularySize = 50257,
  contextLength = 256,
  embeddingDimension = 768,
  attentionHeadsCount = 12,
  layersCount = 12,
  dropoutRate = 0.1,
  queryKeyValueBias = false
)

In [6]:
// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class MultiHeadAttention(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
type TorchTensor = py.Dynamic
def MultiHeadAttention(
  inputDimension: Int,
  outputDimension: Int,
  dropoutProbability: Double,
  contextLength: Int,
  headsCount: Int,
  queryKeyValueBias: Boolean
): py.Dynamic = {
  assert(outputDimension % headsCount == 0, "Output dimension must be a multiple of heads count")
  val headDimension = outputDimension / headsCount
    
  val weightsQuery = torch.nn.Linear(inputDimension, outputDimension, bias = queryKeyValueBias)
  val weightsKey = torch.nn.Linear(inputDimension, outputDimension, bias = queryKeyValueBias)
  val weightsValue = torch.nn.Linear(inputDimension, outputDimension, bias = queryKeyValueBias)
  val outputProjection = torch.nn.Linear(outputDimension, outputDimension)
  val dropout = torch.nn.Dropout(dropoutProbability)
    
  val init = (self: py.Dynamic) => {
    self.register_buffer("mask", torch.triu(torch.ones(contextLength, contextLength), diagonal = 1))
    val mask = self.mask
      
    val forward = (batchedInputs: TorchTensor) => {
      val (batchesCount, tokensCount, tokenDimension) = batchedInputs.shape.as[(Int, Int, Int)]
      val queries = weightsQuery(batchedInputs)
        .view(batchesCount, tokensCount, headsCount, headDimension)
        .transpose(1, 2)
      val keys = weightsKey(batchedInputs)
        .view(batchesCount, tokensCount, headsCount, headDimension)
        .transpose(1, 2)
      val values = weightsValue(batchedInputs)
        .view(batchesCount, tokensCount, headsCount, headDimension)
        .transpose(1, 2)
      val attentionScores = py"$queries @ $keys.transpose(2, 3)"
      attentionScores.masked_fill_(py"$mask.bool()[:$tokensCount, :$tokensCount]", -torch.inf)
      val attentionWeights = dropout(torch.softmax(py"$attentionScores / $outputDimension**0.5", dim = -1))
      outputProjection(
        py"$attentionWeights @ $values"
          .transpose(1, 2)
          .contiguous()
          .view(batchesCount, tokensCount, outputDimension)
      )
    }
    py.Dynamic.global.MultiHeadAttention.forward = forward
  }
  py.Dynamic.global.MultiHeadAttention(init)
}

defined type TorchTensor
defined function MultiHeadAttention

In [7]:
// Workaround to define a class that inherits from a Python class
// Because it mostly uses Python operators, it's implemented fully in Python
py.exec {
  s"""import torch
     |import torch.nn as nn
     |
     |class GELU(nn.Module):
     |  def __init__(self):
     |    super().__init__()
     |
     |  def forward(self, inputs):
     |    return 0.5 * inputs * (
     |      1 + torch.tanh(
     |        torch.sqrt(torch.tensor(2.0 / torch.pi)) * (inputs + 0.044715 * torch.pow(inputs, 3))
     |      )
     |    )
     |""".stripMargin
}
def GELU() = py.Dynamic.global.GELU()

defined function GELU

In [8]:
// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class FeedForward(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
def FeedForward(
  embeddingDimension: Int
): py.Dynamic = {
  val layers = torch.nn.Sequential(
    torch.nn.Linear(embeddingDimension, 4 * embeddingDimension),
    GELU(),
    torch.nn.Linear(4 * embeddingDimension, embeddingDimension)
  )
  val init = (self: py.Dynamic) => {
    val forward = (inputs: TorchTensor) => layers(inputs)
    py.Dynamic.global.FeedForward.forward = forward
  }
  py.Dynamic.global.FeedForward(init)
}

defined function FeedForward

In [9]:
// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class NormalizationLayer(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
def NormalizationLayer(
  embeddingDimension: Int
): py.Dynamic = {
  val epsilon = 1e-5
  val scale = torch.nn.Parameter(torch.ones(embeddingDimension))
  val shift = torch.nn.Parameter(torch.zeros(embeddingDimension))
  val init = (self: py.Dynamic) => {
    val forward = (inputs: TorchTensor) => {
      val mean = inputs.mean(dim = -1, keepdim = true)
      val variance = inputs.`var`(dim = -1, keepdim = true, unbiased = false)
      val normalizedInputs = py"($inputs - $mean) / torch.sqrt($variance + $epsilon)"
      py"$scale * $normalizedInputs + $shift"
    }
    py.Dynamic.global.NormalizationLayer.forward = forward
  }
  py.Dynamic.global.NormalizationLayer(init)
}

defined function NormalizationLayer

In [10]:
import scala.util.chaining._

py.exec {
  s"""import torch.nn as nn
     |
     |class TransformerBlock(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
def TransformerBlock(
  config: GPTConfig
): py.Dynamic = {
  val multiHeadAttention = MultiHeadAttention(
    inputDimension = config.embeddingDimension,
    outputDimension = config.embeddingDimension,
    dropoutProbability = config.dropoutRate,
    contextLength = config.contextLength,
    headsCount = config.attentionHeadsCount,
    queryKeyValueBias = config.queryKeyValueBias
  )
  val feedForward = FeedForward(config.embeddingDimension)
  val normalization1 = NormalizationLayer(config.embeddingDimension)
  val normalization2 = NormalizationLayer(config.embeddingDimension)
  val dropoutShortcut = torch.nn.Dropout(config.dropoutRate)
  val init = (self: py.Dynamic) => {
    val forward = (inputs: TorchTensor) => {
      val shortcut = inputs
      val newShortcut = inputs
        .pipe(normalization1(_))
        .pipe(multiHeadAttention(_))
        .pipe(dropoutShortcut(_))
        .pipe(o => py"$o + $shortcut")
      newShortcut
        .pipe(normalization2(_))
        .pipe(feedForward(_))
        .pipe(dropoutShortcut(_))
        .pipe(o => py"$o + $newShortcut")
    }
    py.Dynamic.global.TransformerBlock.forward = forward
  }
  py.Dynamic.global.TransformerBlock(init)
}

import scala.util.chaining._
defined function TransformerBlock

In [11]:
// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class GPTModel(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
type Model = py.Dynamic
def GPTModel(
  config: GPTConfig
): Model = {
  val tokenEmbeddingLayer = torch.nn.Embedding(config.vocabularySize, config.embeddingDimension)
  val positionEmbeddingLayer = torch.nn.Embedding(config.contextLength, config.embeddingDimension)
  val dropoutEmbeddingLayer = torch.nn.Dropout(config.dropoutRate)
  val transformerBlocks = Seq.fill(config.layersCount)(TransformerBlock(config))
  val transformerBlocksLayer = py"nn.Sequential(*${transformerBlocks.toPythonProxy})"
  val finalNormalizationLayer = NormalizationLayer(config.embeddingDimension)
  val outputLayer = torch.nn.Linear(config.embeddingDimension, config.vocabularySize, bias = false)
  val init = (self: py.Dynamic) => {
    val forward = (batchedInputs: TorchTensor) => {
      val (_, sequenceLength) = batchedInputs.shape.as[(Int, Int)]
      val tokenEmbeddings = tokenEmbeddingLayer(batchedInputs)
      val positionEmbeddings = positionEmbeddingLayer(torch.arange(sequenceLength, device = batchedInputs.device))
      py"$tokenEmbeddings + $positionEmbeddings"
        .pipe(dropoutEmbeddingLayer(_))
        .pipe(transformerBlocksLayer(_))
        .pipe(finalNormalizationLayer(_))
        .pipe(outputLayer(_))
    }
    py.Dynamic.global.GPTModel.forward = forward
  }
  py.Dynamic.global.GPTModel(init)
}

defined type Model
defined function GPTModel

In [12]:
torch.manual_seed(123)
val model = GPTModel(gptConfig)
model.eval()

res12_0: py.Dynamic = <torch._C.Generator object at 0xffff743258d0>
model: Model = GPTModel()
res12_2: py.Dynamic = GPTModel()

In [13]:
def generateTextSimple(
  model: Model,
  maxNewTokens: Int,
  contextLength: Int
)(
  encodedInput: TorchTensor
): TorchTensor =
  LazyList.iterate(encodedInput) { currentEncodedOutput =>
    val croppedInput = py"$currentEncodedOutput[:, -$contextLength:]"
    val logits = py.`with`(torch.no_grad()) { _ =>
      model(croppedInput)
    }
    py"$logits[:, -1, :]"
      .pipe(torch.softmax(_, dim = -1))
      .pipe(torch.argmax(_, dim = -1, keepdim = true))
      .pipe(nextEncodedOutput => torch.cat((currentEncodedOutput, nextEncodedOutput), dim = 1))
  }.drop(maxNewTokens).head

defined function generateTextSimple

In [14]:
type Tokenizer = py.Dynamic
def textToTokenIds(
  text: String, 
  tokenizer: Tokenizer
): TorchTensor = {
  val allowedSpecial = py.Dynamic.global.set(Seq("<|endoftext|>").toPythonProxy)
  val encodedText = tokenizer.encode(text, allowed_special = allowedSpecial)
  torch.tensor(encodedText).unsqueeze(0)
}
    
def tokenIdsToText(
  tokenIds: TorchTensor, 
  tokenizer: Tokenizer
): String =
  tokenizer.decode(tokenIds.squeeze(0).tolist()).as[String]

defined type Tokenizer
defined function textToTokenIds
defined function tokenIdsToText

In [15]:
val tokenizer = tiktoken.get_encoding("gpt2")
val exampleText = "Every effort moves you"
val outputTextIds = generateTextSimple(
  model = model, 
  maxNewTokens = 10, 
  contextLength = gptConfig.contextLength
)(
  encodedInput = textToTokenIds(exampleText, tokenizer)
)
val decodedOutputText = tokenIdsToText(outputTextIds, tokenizer)
println(s"Output text: $decodedOutputText")

Output text: Every effort moves you haw questioning + Vaj370ILSnorth degener dynamicistle


tokenizer: py.Dynamic = <Encoding 'gpt2'>
exampleText: String = "Every effort moves you"
outputTextIds: TorchTensor = tensor([[ 6109,  3626,  6100,   345, 23185, 14085,  1343, 39838, 20167, 45484,
         43588, 25419,  8925, 12535]])
decodedOutputText: String = "Every effort moves you haw questioning + Vaj370ILSnorth degener dynamicistle"

In [16]:
import scala.io.Source

val filePath = "data/the_verdict.txt"
val textData = Source.fromFile(filePath).mkString

import scala.io.Source
filePath: String = "data/the_verdict.txt"
textData: String = """I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)

"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it's going to send the value of my picture 'way up; but I don't think of that, Mr. Rickham--the loss to Arrt is all I think of." The word, on Mrs. Thwing's lips, multiplied its _rs_ as though they were reflected in an endless vista of mirrors. And it was not only the Mrs. Thwings who mourned. Had not the exquisite Hermia Croft, at the last Grafton Gallery show, stopped me before Gisburn's "Moon-dancers" to say, w

In [17]:
val totalCharacters = textData.length
val totalTokens = tokenizer.encode(textData).as[Vector[Int]].length
println("Characters: $totalCharacters")
println("Tokens: $totalTokens")

Characters: $totalCharacters
Tokens: $totalTokens


totalCharacters: Int = 20479
totalTokens: Int = 5145

In [18]:
val trainingRatio = 0.9
val splitIndex = (totalCharacters * trainingRatio).toInt
val (trainingData, validationData) = textData.splitAt(splitIndex)
println(s"Training data size: ${trainingData.size}")
println(s"Validation data size: ${validationData.size}")

Training data size: 18431
Validation data size: 2048


trainingRatio: Double = 0.9
splitIndex: Int = 18431
trainingData: String = """I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)

"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it's going to send the value of my picture 'way up; but I don't think of that, Mr. Rickham--the loss to Arrt is all I think of." The word, on Mrs. Thwing's lips, multiplied its _rs_ as though they were reflected in an endless vista of mirrors. And it was not only the Mrs. Thwings who mourned. Had not the exquisite Hermia Croft, at the last Grafton Gallery show, stopped me before Gisburn's "Moon-dancers" to say, with tears

In [19]:
// Workaround to define a class that inherits from a Python class
py.exec {
  s"""from torch.utils.data import Dataset
     |
     |class GPTDatasetV1(Dataset):
     |  def __init__(self, len, getItem):
     |    self.len = len
     |    self.getItem = getItem
     |  
     |  def __len__(self):
     |    return self.len()
     |
     |  def __getitem__(self, index):
     |    return self.getItem(index)
     |""".stripMargin
}
def GPTDatasetV1(
  text: String,
  tokenizer: Tokenizer,
  maxLength: Int,
  step: Int
): py.Dynamic = {
  val tokens = tokenizer.encode(text).as[Vector[Int]]
  val (inputTokens, outputTokens) = (0 until (tokens.length - maxLength) by step).foldLeft(
    (
      Vector.empty[TorchTensor], 
      Vector.empty[TorchTensor]
    )
  ) {
    case ((inputTokens, outputTokens), i) =>
      val inputChunk = tokens.slice(i, i + maxLength)
      val outputChunk = tokens.slice(i + 1, i + 1 + maxLength)
      (
       inputTokens :+ torch.tensor(inputChunk.toPythonProxy), 
       outputTokens :+ torch.tensor(outputChunk.toPythonProxy)
      )
  }
  val len = () => inputTokens.length
  val getItem = (index: Int) => (inputTokens(index), outputTokens(index))
  py.Dynamic.global.GPTDatasetV1(len, getItem)
}

type DataLoader = py.Dynamic
def createDataLoaderV1(
  text: String, 
  batchSize: Int = 4,
  maxLength: Int = 256,                       
  step: Int = 128, 
  shuffle: Boolean = true, 
  dropLast: Boolean = true,
  numWorkers: Int = 0 
): DataLoader = {
  val tokenizer = tiktoken.get_encoding("gpt2")
  val dataset = GPTDatasetV1(text, tokenizer, maxLength, step)
  torch.utils.data.DataLoader(
    dataset,
    batch_size = batchSize,
    shuffle = shuffle,
    drop_last = dropLast,
    num_workers = numWorkers
  )
}

defined function GPTDatasetV1
defined type DataLoader
defined function createDataLoaderV1

In [20]:
val trainingLoader = createDataLoaderV1(
  text = trainingData, 
  batchSize  = 2, 
  maxLength = gptConfig.contextLength,                       
  step = gptConfig.contextLength, 
  shuffle = true, 
  dropLast = true,
  numWorkers = 0 
)
val validationLoader = createDataLoaderV1(
  text = validationData, 
  batchSize  = 2, 
  maxLength = gptConfig.contextLength,                       
  step = gptConfig.contextLength, 
  shuffle = false, 
  dropLast = false,
  numWorkers = 0 
)

def printBatchShapes(loader: DataLoader) = {
  val loaderBatches = py"[batch for batch in $loader]".as[Seq[Seq[TorchTensor]]]
  loaderBatches.zipWithIndex.foreach { case (Seq(inputBatch, targetBatch), i) =>
    println(s"Batch $i: ${inputBatch.shape} -> ${targetBatch.shape}")
  }
}
println("Training loader batches")
printBatchShapes(trainingLoader)
println()
println("Validation loader batches")
printBatchShapes(validationLoader)

Training loader batches
Batch 0: torch.Size([2, 256]) -> torch.Size([2, 256])
Batch 1: torch.Size([2, 256]) -> torch.Size([2, 256])
Batch 2: torch.Size([2, 256]) -> torch.Size([2, 256])
Batch 3: torch.Size([2, 256]) -> torch.Size([2, 256])
Batch 4: torch.Size([2, 256]) -> torch.Size([2, 256])
Batch 5: torch.Size([2, 256]) -> torch.Size([2, 256])
Batch 6: torch.Size([2, 256]) -> torch.Size([2, 256])
Batch 7: torch.Size([2, 256]) -> torch.Size([2, 256])
Batch 8: torch.Size([2, 256]) -> torch.Size([2, 256])

Validation loader batches
Batch 0: torch.Size([2, 256]) -> torch.Size([2, 256])


trainingLoader: DataLoader = <torch.utils.data.dataloader.DataLoader object at 0xffff51724ad0>
validationLoader: DataLoader = <torch.utils.data.dataloader.DataLoader object at 0xffff52b9d2e0>
defined function printBatchShapes

In [21]:
type Device = py.Dynamic
def calculateBatchLoss(
  model: Model,
  device: Device
)(
  inputBatch: TorchTensor,
  targetBatch: TorchTensor
): TorchTensor = {
  val outputs = model(inputBatch.to(device))
  torch.nn.functional.cross_entropy(outputs.flatten(0, 1), targetBatch.flatten())
}

defined type Device
defined function calculateBatchLoss

In [22]:
def calculateDataLoaderLoss(
  model: Model,
  device: Device
)(
  dataLoader: DataLoader,
  batchesCountOpt: Option[Int] = None
): Double = { 
  val batchesCount = batchesCountOpt match {
    case Some(batchesCount) => batchesCount
    case None => py"len($dataLoader)".as[Int]
  }
  assert(batchesCount > 0, "There were no batches to process")
  println(s"Calculating loss for $batchesCount batches")
  var totalLoss = 0.0
  val batchesIterator = py"iter($dataLoader)"
  var currentBatch = py"next($batchesIterator, None)"
  var currentBatchIndex = 0
  while (
    currentBatchIndex < batchesCount && 
    currentBatch != py.Dynamic.global.None
  ) {
    println(s"Calculating loss for batch: $currentBatchIndex")
    py.local {
      val Seq(inputBatch, targetBatch) = currentBatch.as[Seq[TorchTensor]]
      totalLoss += calculateBatchLoss(model, device)(inputBatch, targetBatch).item().as[Double]
    }
    currentBatchIndex += 1
  }
  totalLoss / batchesCount
}

defined function calculateDataLoaderLoss

In [23]:
val device = torch.device(if (torch.cuda.is_available().as[Boolean]) "cuda" else "cpu")
model.to(device)
py.`with`(torch.no_grad()) { _ =>
  println(s"Calculating training loss")
  val trainingLoss = calculateDataLoaderLoss(model, device)(trainingLoader)
  println(s"Training loss: $trainingLoss")
  println(s"Calculating validation loss")
  val validationLoss = calculateDataLoaderLoss(model, device)(validationLoader)
  println(s"Validation loss: $validationLoss")
}

Calculating training loss
Calculating loss for 9 batches
Calculating loss for batch: 0
Calculating loss for batch: 1
Calculating loss for batch: 2
Calculating loss for batch: 3
Calculating loss for batch: 4
Calculating loss for batch: 5
Calculating loss for batch: 6
Calculating loss for batch: 7
Calculating loss for batch: 8
Training loss: 10.99548308054606
Calculating validation loss
Calculating loss for 1 batches
Calculating loss for batch: 0
Validation loss: 10.979094505310059


device: py.Dynamic = cpu
res23_1: py.Dynamic = GPTModel()